Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']

  del save  # hint to help gc free up memory
  # previous assignment was "copy", not "pointer assignment"
    
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [9]:
# try out "one-hot" in the following code
x = np.arange(5)
y = np.array([0,4,3,3,1,2,1])

# this means adding a new dimension
yy = y[:,None]
print(yy)

# simply transpose a 1D array will not be equaivalent to yy
print('Transpose:', y.transpose())

# first, need to reshape it (-1 means use whatever dataleft to make the shape work)
yt = y.reshape(1, -1)
print('Reshape:', yt)
# then transpose it would give equivalence to yy
print('Reshape transpose:', yt.transpose())

# another way to reshape it
ytt = y.reshape(-1, 1)
# compare: http://stackoverflow.com/questions/10580676/comparing-two-numpy-arrays-for-equality-element-wise
assert np.array_equal(yy, ytt)

# element comparison
r = x == yy
print(r.astype(np.int32))

[[0]
 [4]
 [3]
 [3]
 [1]
 [2]
 [1]]
Transpose: [0 4 3 3 1 2 1]
Reshape: [[0 4 3 3 1 2 1]]
Reshape transpose: [[0]
 [4]
 [3]
 [3]
 [1]
 [2]
 [1]]
[[1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]]


In [10]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  # -1 means automatically infer the shape. here we flatten the index to be 2-D  
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]

  # see code above to understand the logic here
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)

  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [20]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
# Basically we use 1/20 of the original data
train_subset = 10000

# initialize graph, but not open it.
graph = tf.Graph()
# set graph as default, so that tf.* operations are going to apply to that graph.
with graph.as_default():

  # Input data.

  # Load the training, validation and test data into constants that are attached to the graph.
  # This to register symbols?
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    
  # These are the parameters that we are going to be training. 
  # The weight matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  # truncated normal distribution: only specify shape, default mean is 0, std is -1, 1, range: [-1, 1]
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  # not sure why we don't the same randomization technique for the biases (??)
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.

  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.

  # the scores are called "logits", according to the video. this is before softmax.
  # here we define the computation, but not executing it.
  # check shape: train_dataset (# of examples, 28*28) x weights (28*28, 10) = (# of examples, 10),
  # which is the expected shape of results
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # this creates the cross entropy between softmax(logits) and the labels
  # for each training example, return a single cross entropy score. returned results: 1D array
  cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  # this computes the mean of 1D array from last step
  loss = tf.reduce_mean(cross_entropy_loss)
  
  # Optimizer.

  # We are going to find the minimum of this loss using gradient descent.
  # this says: we use GD with learning rate 0.5, to "minimize" the "loss"
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
    
  # predition on the training set, no need to compute logits again since we already have it.  
  train_prediction = tf.nn.softmax(logits)
  # predictions on the validation set. not executing unless we need to
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  # predictions on the test set.
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [21]:
num_steps = 801

# sklearn has accuracy metrics
def accuracy(predictions, labels):
  # np.argmax(...) return the index of the max value on axis=1.
  # axis=0: items. axis=1: class distribution for each item
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# use "graph" as input, start a session.
with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the biases. 
  # Seems we always need to do this.  
  tf.global_variables_initializer().run()
  print('Initialized')
    
  # this is not SGD, we take all training data for each step/iter.  
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy arrays.
    
    # run() takes a list of "runnable" things:
    # optimizer: this is GD that minimizes loss by updating Variables (weights and biases). we don't care about it's return value
    # loss: the total loss, look at it just for fun.
    # predictions: the train predictions tensor, shape (# of examples, class distribution)
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
      
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.214062
Training accuracy: 8.3%
Validation accuracy: 10.1%
Loss at step 100: 2.233637
Training accuracy: 72.1%
Validation accuracy: 70.2%
Loss at step 200: 1.815544
Training accuracy: 75.2%
Validation accuracy: 72.3%
Loss at step 300: 1.588455
Training accuracy: 76.4%
Validation accuracy: 73.2%
Loss at step 400: 1.433186
Training accuracy: 77.3%
Validation accuracy: 73.9%
Loss at step 500: 1.315821
Training accuracy: 78.1%
Validation accuracy: 74.1%
Loss at step 600: 1.222557
Training accuracy: 78.4%
Validation accuracy: 74.2%
Loss at step 700: 1.146251
Training accuracy: 79.0%
Validation accuracy: 74.5%
Loss at step 800: 1.082592
Training accuracy: 79.5%
Validation accuracy: 74.8%
Test accuracy: 82.1%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [24]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  # "placeholder" is like MXNet's DataBatch. only train_dataset needs that.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # note that this is still GD. we do SGD while running
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # the denominator is to make sure we don't take the "leftovers" where we don't have a full batch.
    # basically the leftovers will be discarded and not used in training.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.099401
Minibatch accuracy: 14.8%
Validation accuracy: 15.6%
Minibatch loss at step 500: 1.262932
Minibatch accuracy: 78.1%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 1.353817
Minibatch accuracy: 76.6%
Validation accuracy: 75.9%
Minibatch loss at step 1500: 0.728913
Minibatch accuracy: 83.6%
Validation accuracy: 76.4%
Minibatch loss at step 2000: 0.759294
Minibatch accuracy: 82.8%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 1.097578
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 1.204784
Minibatch accuracy: 78.1%
Validation accuracy: 79.0%
Test accuracy: 86.5%


__Daniel's Note__: If we change `batch_size=10000` and `num_steps=801` and always set `offset=0`, then the SGD example is equivalent to the GD example. I've tested it and run time is similar and results are similar.

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [29]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    
  # for the hidden layer
  h_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  h_biases = tf.Variable(tf.zeros([hidden_nodes]))
  # shape: (batch_size, 768) x (768, 1024) => (batch_size, 1024)
  h_output = tf.matmul(tf_train_dataset, h_weights) + h_biases
    
  # do the ReLU
  relu_output = tf.nn.relu(h_output)

  # for the 2nd layer, which generates the labels
  r_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  r_biases = tf.Variable(tf.zeros([num_labels]))
  # shape: (batch_size, 1024) x (1024, 10) => (batch_size, 10)
  logits = tf.matmul(relu_output, r_weights) + r_biases
    
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(tf_valid_dataset, h_weights) + h_biases),
            r_weights) + r_biases
    )
  test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_dataset, h_weights) + h_biases),
            r_weights) + r_biases
    )

# run it. no code change compared to the previous task

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 380.087769
Minibatch accuracy: 11.7%
Validation accuracy: 25.8%
Minibatch loss at step 500: 28.011425
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 14.002782
Minibatch accuracy: 82.8%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 4.023037
Minibatch accuracy: 91.4%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 4.076619
Minibatch accuracy: 82.8%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 2.943217
Minibatch accuracy: 86.7%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 2.412357
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Test accuracy: 89.4%
